In [2]:
pip install spacy==2.3.2

  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import spacy 

In [3]:
df=pd.read_csv("https://raw.githubusercontent.com/hanzhang0420/Women-Clothing-E-commerce/master/Womens%20Clothing%20E-Commerce%20Reviews.csv")

In [4]:
df.head(3)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses


In [5]:
df.isna().sum()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

In [6]:
df.shape

(23486, 11)

In [7]:
df = df[['Review Text','Recommended IND']].dropna()

In [8]:
df.head(6)

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
5,"I love tracy reese dresses, but this one is no...",0


In [9]:
df.iloc[5]['Review Text']

'I love tracy reese dresses, but this one is not for the very petite. i am just under 5 feet tall and usually wear a 0p in this brand. this dress was very pretty out of the package but its a lot of dress. the skirt is long and very full so it overwhelmed my small frame. not a stranger to alterations, shortening and narrowing the skirt would take away from the embellishment of the garment. i love the color and the idea of the style but it just did not work on me. i returned this dress.'

In [13]:
pip install en_core_web_sm

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement en_core_web_sm (from versions: none)
ERROR: No matching distribution found for en_core_web_sm


In [10]:
nlp=spacy.load("en_core_web_sm")
nlp.pipe_names

['tagger', 'parser', 'ner']

In [11]:
!python -m spacy download en_core_web_sm

  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047114 sha256=7b17faa3f9990dcee6dda0b0fba4336314abbe645ba0cfa9ec61c6024002b895
  Stored in directory: C:\Users\admin\AppData\Local\Temp\pip-ephem-wheel-cache-48a6j6wb\wheels\ee\4d\f7\563214122be1540b5f9197b52cb3ddb9c4a8070808b22d5a84
Successfully built en-core-web-sm
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [11]:
nlp=spacy.load("en_core_web_sm")
nlp.pipe_names

['tagger', 'parser', 'ner']

In [12]:
textcat = nlp.create_pipe( "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"})
nlp.add_pipe(textcat, last=True)
textcat.add_label("POSITIVE")
textcat.add_label("NEGATIVE")
nlp.pipe_names

['tagger', 'parser', 'ner', 'textcat']

In [13]:
textcat.labels

('POSITIVE', 'NEGATIVE')

In [14]:
df['tuples'] = df.apply(lambda row: (row['Review Text'],row['Recommended IND']), axis=1)

In [15]:
df.head(1)

,Review Text,Recommended IND,tuples
0,Absolutely wonderful - silky and sexy and comf...,1,(Absolutely wonderful - silky and sexy and com...


In [16]:
df["tuples"][0]

('Absolutely wonderful - silky and sexy and comfortable', 1)

In [17]:
train = df['tuples'].tolist()

In [18]:
print(train[0])
print(len(train))

('Absolutely wonderful - silky and sexy and comfortable', 1)
22641


In [19]:
texts, labels = zip(*train)

In [20]:
texts[0]

'Absolutely wonderful - silky and sexy and comfortable'

In [21]:
labels[0]

1

In [22]:
cats = []
for y in labels:
    if(bool(y)):
        cats.append({"POSITIVE": True, "NEGATIVE":False})
    else:
        cats.append({"POSITIVE": False, "NEGATIVE":True})

In [23]:
TrainX = texts
TrainY = cats

In [24]:
n_texts=23486

In [25]:
len(TrainX)

22641

In [26]:
len(TrainY)

22641

In [27]:
train_data = list(zip(TrainX,[{'cats': cats} for cats in TrainY]))

In [28]:
train_data[0]

('Absolutely wonderful - silky and sexy and comfortable',
 {'cats': {'POSITIVE': True, 'NEGATIVE': False}})

In [29]:
len(train_data)

22641

In [30]:
len(train_data)

22641

In [31]:
n_iter=10

In [32]:
from spacy.util import minibatch, compounding
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes): 
    optimizer = nlp.begin_training()
    
    print("Training the model...")

    for i in range(n_iter):
        print("Epoch : {} ".format(i))
        losses = {}
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)

Training the model...
Epoch : 0 
Epoch : 1 
Epoch : 2 
Epoch : 3 
Epoch : 4 
Epoch : 5 
Epoch : 6 
Epoch : 7 
Epoch : 8 
Epoch : 9 


In [34]:
nlp.to_disk("sentiment")
nlp = spacy.load("sentiment")

In [35]:
import cv2
import pytesseract as pt
import numpy as np

In [36]:
img = cv2.imread('imagetext.png')
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray, img_bin = cv2.threshold(gray,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
gray = cv2.bitwise_not(img_bin)

In [38]:
pt.pytesseract.tesseract_cmd=r'C:\Users\admin\AppData\Local\Tesseract-OCR\tesseract.exe'
kernel = np.ones((2, 1), np.uint8)
img = cv2.erode(gray, kernel, iterations=1)
img = cv2.dilate(img, kernel, iterations=1)
out_below = pt.image_to_string(img)
print("OUTPUT: ", out_below)

OUTPUT: Worst Product, I hate it


In [39]:
test_text = out_below
doc=(nlp(test_text))
doc.cats

{'POSITIVE': 0.059504084289073944, 'NEGATIVE': 0.9404958486557007}